In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Apr 24 21:24:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   27C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install peft datasets transformers trl accelerate bitsandbytes -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 15.2 MB/s eta 0:00:00


## Mount Drive

In [ ]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Libraries

In [ ]:
# Import libraries
import pandas as pd
import os
from google.colab import userdata
from tqdm import tqdm
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, DataCollatorForLanguageModeling, pipeline, logging

# Fine tuning frameworks
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Dataset processing
from datasets import Dataset, load_dataset
device = torch.device("cuda")

# Access token
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

# Model Finetuning

## Define Train and Validation Data Paths

In [ ]:
# Data Path
train_data = '/content/drive/MyDrive/childes-train.txt'
val_data = '/content/drive/MyDrive/childes-val.txt'

## Set up Quantization

In [ ]:
# define model id
model_id = "google/gemma-2b"

# Quantization parameters
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

## Set Up Tokenizer and model

In [ ]:
# Set up tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

## Generation with Base Model Before Finetuning

In [ ]:
# Read the test data into a pandas DataFrame
test = pd.read_csv('/content/drive/MyDrive/feedback_test.csv', dtype={'prompt': str})

# Specify GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Iterate over each prompt in the list
for idx, row in tqdm(test.iterrows(), desc="Generating Responses", total=len(test)):
    prompt = row['prompt']

    # Encode the prompt with EOS token
    input_ids = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate responses with top-k sampling
    output = model.generate(
        **input_ids,
        max_new_tokens=10,
        num_return_sequences=5,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50
    )

    #print(tokenizer.decode(output[0]))

    # Decode and store the generated responses (temporary list)
    generated_responses = []
    for i in range(output.shape[0]):
        generated_response = tokenizer.decode(output[i], skip_special_tokens=True, clean_up_tokenization_spaces=False)
        generated_responses.append(generated_response)

    # Last utterance by C (for overlap calculation)
    last_utterance_by_c = prompt.split('\n')[-2]

    # Re-rank based on overlap with last utterance by C
    def overlap_score(response):
        response_words = set(response.split())
        last_utterance_words = set(last_utterance_by_c.split())
        return len(response_words.intersection(last_utterance_words))

    ranked_responses = sorted(generated_responses, key=lambda x: overlap_score(x), reverse=True)

    # Store the top 5 ranked responses
    for i in range(5):
        processed_response = ranked_responses[i].replace(prompt.replace('', ''), '').replace('\n', ' ')
        test.at[idx, f'generated_response_{i+1}'] = processed_response

# Save the DataFrame to a CSV file
test.to_csv('gemma_base_feedback_generated_responses.csv', index=False)

Using device: cuda


Generating Responses: 100%|██████████| 80/80 [01:38<00:00,  1.23s/it]


## Lora congif

In [ ]:
# Low rank adaptation
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    #lora_dropout=0.1,
    #lora_alpha = 16,
    task_type = "CAUSAL_LM",
)

## Load and Tokenize Dataset


In [ ]:
# load dataset
dataset = load_dataset("text", data_files={"train": train_data, "validation":val_data})

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset["train"]

Dataset({
    features: ['text'],
    num_rows: 770212
})

In [ ]:
dataset["train"][100]

{'text': "A: you've got lots of things to play with. you don't need it. look. you've got this lorry here. you'll hafta make do with this one. okay? you've got a very good memory because you're sposta have forgotten about that."}

In [ ]:
# Function to tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=1024)

In [ ]:
# Apply function
tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=None, remove_columns=["text"])

Map:   0%|          | 0/770212 [00:00<?, ? examples/s]

Map:   0%|          | 0/203319 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets["train"][1]

{'input_ids': [2, 235288, 235292, 590, 1554, 674, 974, 235265],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# block_size = tokenizer.model_max_length
block_size = 1024

# Preprocessing function to group texts
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
## Speed-up the preprocessing by using multiprocessing
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    #batch_size=1000,
    num_proc=None,
)

Map:   0%|          | 0/770212 [00:00<?, ? examples/s]

Map:   0%|          | 0/203319 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(lm_datasets["train"][0]["input_ids"])

"<bos>A: right. if I could find somewhere to sit Dominic. what're you playing with?<bos>C: I play that one.<bos>A: what? with that one?<bos>C: digger.<bos>A: oh. oh.<bos>C: digger. oh oh. vroom.<bos>A: I didn't know diggers made that kind of noise.<bos>C: vroom.<bos>A: vroom. do you know? diggers don't go on tracks Dominic.<bos>C: digger track.<bos>A: they don't. you find them on building sites. shall we put these bricks in your brick truck? hm? so Mummy can have somewhere to sit down. are you gonna going to help me?<bos>C: mhm.<bos>A: yes?<bos>C: I not.<bos>A: you're not?<bos>C: mhm.<bos>A: that's not very nice is it? come on. put them in the brick truck.<bos>C: sister gone sleep.<bos>A: she's not asleep.<bos>C: more sleep.<bos>A: she was asleep. I hadta wake her up didn't I?<bos>C: mhm.<bos>A: mhm. oh. oh. broken.<bos>C: Mummy.<bos>A: oh. isn't broken. we're alright. what was it?<bos>C: um Pooh Bear.<bos>A: Pooh Bear. are you gonna going to put those in there as well?<bos>C: mhm. the

In [ ]:
# create data collator
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
tokenizer.padding_side = "right"

## Define Training Arguments

In [ ]:
# Define training arguments
training_arguments = TrainingArguments(
    output_dir = "/content/drive/MyDrive/tuned-gemma",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=50,
    load_best_model_at_end=True,
    save_strategy="steps",
    save_total_limit=1,
    metric_for_best_model="eval_loss",
    logging_steps=50,
    save_steps=50,
    #optim="paged_adamw_32bit",
    fp16=True,
)

## Train Model

In [ ]:
# SFT trainer
trainer = SFTTrainer(
    model,
    tokenizer=tokenizer,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    dataset_text_field='text',
    args = training_arguments,
    peft_config=lora_config,
    data_collator=data_collator,
    max_seq_length=512,

)

In [ ]:
# Set the environment variable to limit GPU memory allocation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("fine-tuned-gemma")
trainer.save_state()

# Model Generation

### Merge fine tuned model with saved model

In [ ]:
# Provide the location of the base model and the adopter.
base_model = "google/gemma-2b"
new_model = '/content/drive/MyDrive/fine-tuned-gemma'

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
 #  Load the base model
base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
#  Load and merge adopter with base model
model = PeftModel.from_pretrained(base_model_reload, new_model)

### Test generation

In [ ]:
# Test generation with a single prompt

prompt = '''C: there .
A: thank you .
C: look my sweeper broke .
A: it broke ?
A: what happened to it ?
C: what happen it ?
A:'''

inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=10, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


C: there .
A: thank you .
C: look my sweeper broke .
A: it broke ?
A: what happened to it ?
C: what happen it ?
A: it broke ?
C: it broke .



### Generate with Test Data

In [ ]:
# Read the test data into a pandas DataFrame
import pandas as pd
test = pd.read_csv('/content/drive/MyDrive/rephrase_test.csv', dtype={'prompt': str})

# Specify GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Iterate over each prompt in the list
for idx, row in tqdm(test.iterrows(), desc="Generating Responses", total=len(test)):
    prompt = row['prompt']

    # Encode the prompt with EOS token
    model = PeftModel.from_pretrained(base_model_reload, new_model)
    tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"

    input_ids = tokenizer(prompt, return_tensors='pt', add_special_tokens=True)["input_ids"].to(device)

    # Generate responses with top-k sampling
    output = model.generate(
        input_ids=input_ids,
        max_new_tokens=7,
        num_return_sequences=5,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50
    )

    # Decode and store the generated responses (temporary list)
    generated_responses = []
    for i in range(output.shape[0]):
        generated_response = tokenizer.decode(output[i], skip_special_tokens=True, clean_up_tokenization_spaces=False)
        generated_responses.append(generated_response)

    # Last utterance by C (for overlap calculation)
    last_utterance_by_c = prompt.split('\n')[-2]

    # Re-rank based on overlap with last utterance by C
    def overlap_score(response):
        response_words = set(response.split())
        last_utterance_words = set(last_utterance_by_c.split())
        return len(response_words.intersection(last_utterance_words))

    ranked_responses = sorted(generated_responses, key=lambda x: overlap_score(x), reverse=True)

    # Store the top 5 ranked responses
    for i in range(5):
        processed_response = ranked_responses[i].replace(prompt.replace('', ''), '').replace('\n', ' ')
        test.at[idx, f'generated_response_{i+1}'] = processed_response

# Save the DataFrame to a CSV file
test.to_csv('rephrase_generated.csv', index=False)

Using device: cuda


Generating Responses: 100%|██████████| 69/69 [01:58<00:00,  1.72s/it]


### Citation

The code used in this work is available in the following Hugging Face repositories:
- [Hugging Face](https://huggingface.co/blog/4bit-transformers-bitsandbytes)
- [Hugging Face](https://huggingface.co/docs/transformers/en/tasks/language_modeling)
